<a href="https://colab.research.google.com/github/bo7/django/blob/master/JuBo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install alpha_vantage

  Created wheel for alpha-vantage: filename=alpha_vantage-2.1.3-cp36-none-any.whl size=14872 sha256=b379154e4e1f7eb479053cf38beb5534110cb6e908bf61889b03303b6d1b3382
  Stored in directory: /root/.cache/pip/wheels/d4/75/c2/483b0a988fd965b1128c1396a0ffadfb8903bd2bb569220048
Successfully built alpha-vantage


In [0]:
import datetime
import pandas as pd
import matplotlib.pyplot as plt
from alpha_vantage.timeseries import TimeSeries
from pprint import pprint
import numpy as np


In [0]:
pd.set_option('display.expand_frame_repr', False)
ts = TimeSeries(key='7TFOURL7IICYG54G',output_format='pandas')
data, meta_data = ts.get_daily(symbol='MSFT', outputsize='full')
data.columns = ['open', 'high', 'low', 'close', 'volume']
# add average 
data.sort_index(axis=0, inplace=True)
data['average'] = data['close']    #(data['high']+data['low'])/2 # nur auf den Close Kurs berechnen 
for i in range(3, 200):
    data['MA' + str(i)] = data.iloc[:,5].rolling(window=i).mean().round(2)
# data['MA5'] = pd.rolling_mean(Exchange,5)
#data.reset_index()

#print(data.head(30))
#print(data.tail(30))
#print(data.shape)

In [0]:
data.reset_index()
plt.figure(figsize=(30,10)) #30, 50, 90, 120 150, 199
ax = plt.gca()
data.plot(y='MA199', use_index=True, ax=ax)
data.plot(y='close', use_index=True, ax=ax)

plt.show()

#berechnen close - max
# buy hold sell wait -> dummyvariablen 

In [0]:

plt.figure(figsize=(30,10))
ax1 = plt.subplot(2, 3, 1)
data.plot(y='MA30', use_index=True, ax=ax1)
data.plot(y='close', use_index=True, ax=ax1)

ax1.set_title("MA30")


ax2 = plt.subplot(2, 3, 2)
data.plot(y='MA60', use_index=True, ax=ax2)
data.plot(y='close', use_index=True, ax=ax2)
ax2.set_title("MA60")

ax3 = plt.subplot(2, 3, 3)
data.plot(y='MA90', use_index=True, ax=ax3)
data.plot(y='close', use_index=True, ax=ax3)
ax3.set_title("MA90")

ax4 = plt.subplot(2, 3, 4)
data.plot(y='MA120', use_index=True, ax=ax4)
data.plot(y='close', use_index=True, ax=ax4)
ax4.set_title("MA120")

ax5 = plt.subplot(2, 3, 5)
data.plot(y='MA150', use_index=True, ax=ax5)
data.plot(y='close', use_index=True, ax=ax5)
ax5.set_title("MA150")

ax6 = plt.subplot(2, 3, 6)
data.plot(y='MA180', use_index=True, ax=ax6)
data.plot(y='close', use_index=True, ax=ax6)
ax6.set_title("MA180")

plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=0.5, wspace=0.2)
plt.show()

In [0]:
data_close_MAx = data.loc[:, 'MA3':]
#print(data.tail())

data_close_MAx = data_close_MAx * (-1)
data_close_MAx =  data_close_MAx.add(data['close'], axis=0)
print(data_close_MAx.tail())

             MA3   MA4   MA5   MA6   MA7   MA8   MA9  MA10  MA11  MA12  MA13  MA14  MA15  MA16  MA17  MA18  MA19  MA20  MA21  MA22  MA23  MA24  MA25  MA26  MA27  MA28  MA29  MA30  MA31  MA32   MA33   MA34   MA35   MA36   MA37   MA38   MA39   MA40   MA41   MA42  ...  MA160  MA161  MA162  MA163  MA164  MA165  MA166  MA167  MA168  MA169  MA170  MA171  MA172  MA173  MA174  MA175  MA176  MA177  MA178  MA179  MA180  MA181  MA182  MA183  MA184  MA185  MA186  MA187  MA188  MA189  MA190  MA191  MA192  MA193  MA194  MA195  MA196  MA197  MA198  MA199
date                                                                                                                                                                                                                                                                  ...                                                                                                                                                                                             

In [0]:
df = data_close_MAx
#df['MA3'] = 0
for i in range(3,200):
#conditions = [
#      (df['MA'+str(i)] > 0),
#      (df['MA'+str(i)] < 0) ,
#      (df['MA'+str(i)] == 0) &  (df.loc[:, 'MA'+str(i)].shift(1) < 0),
#      (df['MA'+str(i)] == 0) &  (df.loc[:, 'MA'+str(i)].shift(1) > 0),
#      (df['MA'+str(i)] == 0) &  (df.loc[:, 'MA'+str(i)].shift(1) == 0)
#      ]
conditions = [
        ( (df['high'] >= df['MA'+str(i)]) or  (df['low'] > df['MA'+str(i)])  ),      

]
  choices = ['in', 'out', 'in', 'out', 'out']
  df['MA'+str(i)+'ind'] = np.select(conditions, choices, default= np.NaN)
  conditions2 = [
      (df['MA'+str(i)+'ind'] == 'in') & (df.loc[:, 'MA'+str(i)+'ind'].shift(1) == 'out'),
      (df['MA'+str(i)+'ind'] == 'in') & (df.loc[:, 'MA'+str(i)+'ind'].shift(1) == 'in'),
      (df['MA'+str(i)+'ind'] == 'out') & (df.loc[:, 'MA'+str(i)+'ind'].shift(1) == 'in'),
      (df['MA'+str(i)+'ind'] == 'out') & (df.loc[:, 'MA'+str(i)+'ind'].shift(1) == 'out')
      ]
  choices2 = ['buy', 'hold', 'sell', 'wait']
  df['MA'+str(i)+'dummy'] = np.select(conditions2, choices2, default= np.NaN)

#df.loc[df['MA3ind'] == 'FUCK']
for i in range(3,200):
  #pprint(df['MA'+str(i)+'dummy'].unique())
  print(df.groupby('MA'+str(i)+'dummy').count())


# WENN(G6<0;"out";WENN(G6>0;"in";WENN(UND(G6=0;G5<0);"in";WENN(UND(G6=0;G5>0);"out";"out"))))

# =WENN(UND(H6="out";H7="in");"buy";WENN(UND(H6="in";H7="in");"hold";WENN(UND(H6="in";H7="out");"sell";WENN(UND(H6="out";H7="out");"wait";"NA"))))

# =IF(AND(J11="out";J12="in");"buy";IF(AND(J11="in";J12="in");"hold";IF(AND(J11="in";J12="out");"sell";IF(AND(J11="out";J12="out");"wait";"NA"))))

           MA3   MA4   MA5   MA6   MA7   MA8   MA9  MA10  MA11  MA12  MA13  MA14  MA15  MA16  MA17  MA18  MA19  MA20  MA21  MA22  MA23  MA24  MA25  MA26  MA27  MA28  MA29  MA30  MA31  MA32  MA33  MA34  MA35  MA36  MA37  MA38  MA39  MA40  MA41  MA42  ...  MA180ind  MA180dummy  MA181ind  MA181dummy  MA182ind  MA182dummy  MA183ind  MA183dummy  MA184ind  MA184dummy  MA185ind  MA185dummy  MA186ind  MA186dummy  MA187ind  MA187dummy  MA188ind  MA188dummy  MA189ind  MA189dummy  MA190ind  MA190dummy  MA191ind  MA191dummy  MA192ind  MA192dummy  MA193ind  MA193dummy  MA194ind  MA194dummy  MA195ind  MA195dummy  MA196ind  MA196dummy  MA197ind  MA197dummy  MA198ind  MA198dummy  MA199ind  MA199dummy
MA3dummy                                                                                                                                                                                                                                                  ...                                                     

In [0]:
df.iloc[df.index.get_loc('MA3')-1]

KeyError: ignored